# The Street View House Numbers (SVHN) Dataset

## Содержание

1. [Подключение необходимых библиотек и загрузка датасета](#Подключение-необходимых-библиотек-и-загрузка-датасета) 
2. [Описание датасета](#Описание-датасета)
    1. [Структура](#Структура)
    2. [Вывод изображений из датасета](#Вывод-изображений-из-датасета)
3. [Задание №1](#Задание-№1)
    1. [Задача бинарной классификации](#Задача-бинарной-классификации)
        1. [Предобработка данных](#Предобработка-данных)
        2. [Обучение модели](#Обучение-модели)
        3. [Прогнозирование](#Прогнозирование)
        4. [Точность полученной модели](#Точность-полученной-модели)
        5. [Обучение множества моделей с гиперпараметром от k=1 до k=max](#Обучение-множества-моделей-с-гиперпараметром-от-k=1-до-k=max)
    2. [Задача множественной классификации](#Задача-множественной-классификации)
        1. [Предобработка данных](#Предобработка-данных.)
        2. [Обучение модели](#Обучение-модели.)
        3. [Прогнозирование](#Прогнозирование.)
        4. [Точность полученной модели](#Точность-полученной-модели.)
        5. [Обучение множества моделей с гиперпараметром от k=1 до k=max](#Обучение-множества-моделей-с-гиперпараметром-от-k=1-до-k=max.)
4. [Задание №2](#Задание-№2)
    1. [Поиск оптимального значения гиперпараметра методом кросс-валидации](#Поиск-оптимального-значения-гиперпараметра-методом-кросс-валидации)
    2. [Прогноз на тестовых данных с моделью, обученной при оптимальном значении k](#Прогноз-на-тестовых-данных-с-моделью,-обученной-при-оптимальном-значении-k)
    3. [Точность модели при оптимальном k](#Точность-модели-при-оптимальном-k)

## Подключение необходимых библиотек и загрузка датасета

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from dataset import load_svhn
from sklearn.model_selection import train_test_split

%matplotlib notebook
#%matplotlib inline

train_X, train_y, test_X, test_y = load_svhn("data", max_train=1000, max_test=100)

## Описание датасета
### Структура
Структура датасета следующая:
1. `train_X[i]` — i-ое изображение датасета;
2. `train_X[i][row][col]` — вернет список из 3 элементов, где каждый элемент является интенсивностью цвета r, g, b соответственно, row*col пикселя i-ого изображения;

Подробную информацию о датасете можно найти [здесь](http://ufldl.stanford.edu/housenumbers/).

In [ ]:
train_X.shape

### Вывод изображений из датасета

In [ ]:
print(train_y[:36].reshape(6, 6))

plot_index = 1
for i in range(36):
    plt.subplot(6, 6, plot_index)
    image = train_X[i]
    plt.imshow(image.astype(np.uint8))
    plt.axis('off')
    plot_index += 1

## Задание №1
### Постановка задачи
Обучить модель и получить прогнозы для бинарной и мультиклассовой классификации набора данных street view house numbers. Вычислить значения метрик точности (accuracy, precision, recall, fscore) при трех разных значениях гиперпараметра k.
### Задача бинарной классификации
#### Предобработка данных

In [ ]:
binary_train_mask = (train_y == 0) | (train_y == 9)
binary_train_X = train_X[binary_train_mask]
binary_train_y = train_y[binary_train_mask]

binary_test_mask = (test_y == 0) | (test_y == 9)
binary_test_X = test_X[binary_test_mask]
binary_test_y = test_y[binary_test_mask]


# Преобразование в одномерный массив [num_samples, 32*32*3]
binary_train_X = binary_train_X.reshape(binary_train_X.shape[0], -1)
binary_test_X = binary_test_X.reshape(binary_test_X.shape[0], -1)

#### Обучение модели

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

model = KNeighborsClassifier(n_neighbors=4)
model.fit(binary_train_X, binary_train_y)

#### Прогнозирование

In [ ]:
y_pred = model.predict(binary_test_X)

print(f"{binary_test_y} -> Метки тестового набора")
print(f"{y_pred} -> Прогнозы для тестового набора")

#### Точность полученной модели

In [ ]:
from sklearn.metrics import precision_recall_fscore_support

print('accuracy: {:.2f}'.format(model.score(binary_test_X, binary_test_y)))

print(
    "precision: {}\
    \nrecall: {}\
    \nfscore: {}\
    \nsupport: {}".format(*precision_recall_fscore_support(binary_test_y, y_pred, average='macro'))
)

#### Обучение множества моделей с гиперпараметром от k=1 до k=max

In [ ]:
accuracies, precision, recall, fscore = [], [], [], []

for k in range(len(binary_train_X)):
    model = KNeighborsClassifier(n_neighbors=k+1)
    model.fit(binary_train_X, binary_train_y)
    y_pred = model.predict(binary_test_X)
    
    # Вычисление ошибок
    accuracies.append(model.score(binary_test_X, binary_test_y))
    pr, rec, fs, sup = precision_recall_fscore_support(binary_test_y, y_pred, average='macro', warn_for=tuple())

    precision.append(pr)
    recall.append(rec)
    fscore.append(fs)

plt.plot(np.arange(1, 122), accuracies, label="score")
plt.plot(np.arange(1, 122), precision, label="precision")
plt.plot(np.arange(1, 122), recall, label="recall")
plt.plot(np.arange(1, 122), fscore, label="fscore")

plt.legend(loc='center right', frameon=True, ncol=2)
plt.show()

### Задача множественной классификации

#### Предобработка данных.

In [ ]:
# Преобразование в одномерный массив [num_samples, 32*32*3]
mult_train_X = train_X.reshape(train_X.shape[0], -1)
mult_test_X = test_X.reshape(test_X.shape[0], -1)

#### Обучение модели.

In [ ]:
model = KNeighborsClassifier(n_neighbors=4)

model.fit(mult_train_X, train_y)

#### Прогнозирование.

In [ ]:
y_pred = model.predict(mult_test_X)

print(f"Метки тестового набора:\n{test_y}\n")
print(f"Прогнозы для тестового набора:\n{y_pred}")

#### Точность полученной модели.

In [ ]:
print('accuracy: {:.2f}'.format(model.score(mult_test_X, test_y)))

print(
    "precision: {}\
    \nrecall: {}\
    \nfscore: {}\
    \nsupport: {}".format(*precision_recall_fscore_support(test_y, y_pred, average='macro'))
)

#### Обучение множества моделей с гиперпараметром от k=1 до k=max.

In [ ]:
# Осторожно! Длительные вычисления. На моем железе ячейка вычисляется ~1 мин. 10 сек.

accuracies, precision, recall, fscore = [], [], [], []

for k in range(len(mult_train_X)):
    model = KNeighborsClassifier(n_neighbors=k+1)
    model.fit(mult_train_X, train_y)
    y_pred = model.predict(mult_test_X)
    
    # Вычисление ошибок
    accuracies.append(model.score(mult_test_X, test_y))
    pr, rec, fs, sup = precision_recall_fscore_support(test_y, y_pred, average='macro', warn_for=tuple())

    precision.append(pr)
    recall.append(rec)
    fscore.append(fs)

plt.plot(np.arange(1, 1001), accuracies, label="score")
plt.plot(np.arange(1, 1001), precision, label="precision")
plt.plot(np.arange(1, 1001), recall, label="recall")
plt.plot(np.arange(1, 1001), fscore, label="fscore")

plt.legend(loc='upper right', frameon=True, ncol=2)
plt.show()

## Задание №2
### Постановка задачи
Реализовать метод кросс-валидации для подбора оптимального значения гиперпараметра k из массива k_choices = [1, 2, 3, 5, 8, 10, 15, 20, 25, 50]. Разбить датасет на части с помощью функции [StratifiedKFold](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedKFold.html) (from sklearn.model_selection), вычислить средние метрики (accuracy, precision, recall, fscore) для каждого k из массива k_choices на валидационных данных. Сделать прогноз на тестовых данных с моделью, обученной при оптимальном значении k.
### Поиск оптимального значения гиперпараметра методом кросс-валидации

In [ ]:
import statistics as stat
from sklearn.model_selection import StratifiedKFold

k_choices = [1, 2, 3, 5, 8, 10, 15, 20, 25, 50]
accuracies = []

folds = StratifiedKFold(n_splits=5)
for k in k_choices:
    accuracy = []
    for train_index, test_index in folds.split(mult_train_X, train_y):
        X_train_folds, y_train_folds = mult_train_X[train_index], train_y[train_index]
        X_test_fold, y_test_fold = mult_train_X[test_index], train_y[test_index]
        
        model = KNeighborsClassifier(n_neighbors=k)
        model.fit(X_train_folds, y_train_folds)
        
        accuracy.append(model.score(X_test_fold, y_test_fold))
    accuracies.append(stat.mean(accuracy))

best_k = k_choices[accuracies.index(max(accuracies))]

In [ ]:
plt.plot(k_choices, accuracies, k_choices, accuracies, '.', label="score")
print(f"Best k = {best_k}")

### Прогноз на тестовых данных с моделью, обученной при оптимальном значении k

In [ ]:
model =  KNeighborsClassifier(n_neighbors=best_k)
model.fit(mult_train_X, train_y)

y_pred = model.predict(mult_test_X)

print(f"Метки тестового набора:\n{test_y}\n")
print(f"Прогнозы для тестового набора:\n{y_pred}")

### Точность модели при оптимальном k

In [ ]:
print('accuracy: {:.2f}'.format(model.score(mult_test_X, test_y)))

print(
    "precision: {}\
    \nrecall: {}\
    \nfscore: {}\
    \nsupport: {}".format(*precision_recall_fscore_support(test_y, y_pred, average='macro', warn_for=tuple()))
)